In [9]:
# import everything
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import KMeans


In [11]:
# read data

train = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# get data and labels
train_data = train.drop(['Target Variable (Discrete)'], axis=1)
train_labels = train['Target Variable (Discrete)']

# standardize data
scaler = StandardScaler()
scaler.fit(train_data)
train_data_ = scaler.transform(train_data)
train_data, val_data, train_labels, val_labels = train_test_split(train_data_, train_labels, test_size=0.2, random_state=42)
test_data = scaler.transform(test_data)


# print shapes
train_data.shape, train_labels.shape, val_data.shape, val_labels.shape, test_data.shape


((795, 24), (795,), (199, 24), (199,), (426, 24))

In [8]:
np.bincount(train_labels) + np.bincount(val_labels, minlength=18)

array([249, 488, 109,   3,   3,  41,  70,   5,   7,   2,   1,   1,   1,
         3,   5,   4,   1,   1])

In [16]:
# knn imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
all_imp = imputer.fit_transform(train_data_)
train_data_imp = imputer.transform(train_data)
val_data_imp = imputer.transform(val_data)
test_data_imp = imputer.transform(test_data)
# train_data_imp.shape
for i in train_data_imp:
    for j in i:
        if np.isnan(j):
            print('nan')

In [17]:
# feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# chi2

chi2_selector = SelectKBest(chi2, k=10)
chi2_selector.fit(train_data_imp, train_labels)
chi2_support = chi2_selector.get_support()
chi2_feature = train_data_imp.loc[:,chi2_support].columns.tolist()
print(str(len(chi2_feature)), 'selected features')

ValueError: Input X must be non-negative.

In [80]:
# knn classifier

# grid search
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 25)}
knn_gscv = GridSearchCV(knn, param_grid, cv=5)
knn_gscv.fit(train_data_imp, train_labels)
nn = knn_gscv.best_params_

# find accuracy
knn = KNeighborsClassifier(n_neighbors=nn['n_neighbors'])
knn.fit(train_data_imp, train_labels)
knn_pred = knn.predict(val_data_imp)
accuracy_score(val_labels, knn_pred)
f1_score(val_labels, knn_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.3774690855196913

In [82]:
# softmax classifier

logreg = LogisticRegression(max_iter=10000)
logreg.fit(train_data_imp, train_labels)
logreg_pred = logreg.predict(val_data_imp)
accuracy_score(val_labels, logreg_pred)
f1_score(val_labels, logreg_pred, average='macro')

0.3572219315076458

In [18]:
# try with random forests

# grid search
rf = RandomForestClassifier()
param_grid = {'n_estimators': np.arange(1, 250)}
rf_gscv = GridSearchCV(rf, param_grid, cv=5)
rf_gscv.fit(train_data_imp, train_labels)
nn = rf_gscv.best_params_

est = nn['n_estimators']
# find accuracy
rf = RandomForestClassifier(n_estimators=nn['n_estimators'])
rf.fit(train_data_imp, train_labels)
rf_pred = rf.predict(val_data_imp)
accuracy_score(val_labels, rf_pred)
f1_score(val_labels, rf_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.4727839505189698

In [85]:
# SVC

# grid search
svc = SVC()
param_grid = {'C': np.arange(1, 25)}
svc_gscv = GridSearchCV(svc, param_grid, cv=5)
svc_gscv.fit(train_data_imp, train_labels)
nn = svc_gscv.best_params_

# find accuracy
svc = SVC(C=nn['C'])
svc.fit(train_data_imp, train_labels)
svc_pred = svc.predict(val_data_imp)
accuracy_score(val_labels, svc_pred)
f1_score(val_labels, svc_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.4067353865231955

In [86]:
# decision tree

# grid search
dt = DecisionTreeClassifier()
param_grid = {'max_depth': np.arange(1, 25)}
dt_gscv = GridSearchCV(dt, param_grid, cv=5)
dt_gscv.fit(train_data_imp, train_labels)
nn = dt_gscv.best_params_

# find accuracy
dt = DecisionTreeClassifier(max_depth=nn['max_depth'])
dt.fit(train_data_imp, train_labels)
dt_pred = dt.predict(val_data_imp)
accuracy_score(val_labels, dt_pred)
f1_score(val_labels, dt_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.2934220792557482

In [87]:
# gradient boosting

# grid search
gb = GradientBoostingClassifier()
param_grid = {'n_estimators': np.arange(1, 25)} 
gb_gscv = GridSearchCV(gb, param_grid, cv=5)
gb_gscv.fit(train_data_imp, train_labels)
nn = gb_gscv.best_params_

# find accuracy
gb = GradientBoostingClassifier(n_estimators=nn['n_estimators'])
gb.fit(train_data_imp, train_labels)
gb_pred = gb.predict(val_data_imp)
accuracy_score(val_labels, gb_pred)
f1_score(val_labels, gb_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.40401242807940496

In [88]:
# neural network

# grid search
nn = MLPClassifier(max_iter=10000)
param_grid = {'hidden_layer_sizes': np.arange(1, 25)}
nn_gscv = GridSearchCV(nn, param_grid, cv=5)
nn_gscv.fit(train_data_imp, train_labels)
nn = nn_gscv.best_params_

# find accuracy
nn = MLPClassifier(hidden_layer_sizes=nn['hidden_layer_sizes'], max_iter=10000)
nn.fit(train_data_imp, train_labels)
nn_pred = nn.predict(val_data_imp)
accuracy_score(val_labels, nn_pred)
f1_score(val_labels, nn_pred, average='macro')

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.39186202686202687

In [ ]:
# cross-validation to output f1-score
from sklearn.model_selection import cross_val_score

cross_val_score(knn, train_data_imp, train_labels, cv=5)
cross_val_score(logreg, train_data_imp, train_labels, cv=5)
cross_val_score(rf, train_data_imp, train_labels, cv=5)
cross_val_score(svc, train_data_imp, train_labels, cv=5)
cross_val_score(dt, train_data_imp, train_labels, cv=5)

In [89]:
# final: random forests

rf = RandomForestClassifier(n_estimators=est)
all_labels = np.concatenate((train_labels, val_labels))
rf.fit(all_imp, all_labels)
rf_pred = rf.predict(test_data_imp)

# write to csv
rf_pred = pd.DataFrame(np.array([(i + 1, v) for i, v in enumerate(rf_pred)]))
rf_pred.columns = ['Id', 'Category']
rf_pred.to_csv('rf_pred.csv', index=False)


In [5]:
est

63